In [1]:
import os
import tqdm
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from tfcf.metrics import mae
from tfcf.metrics import rmse
from tfcf.datasets import ml1m
from tfcf.config import Config
from tfcf.models.svd import SVD
# from tfcf.models.svd import SVDPP
# from sklearn.model_selection import train_test_split

dir_ = '../../data/'
# file_name = 'normalized_minmax_filter_track_5_user_100.csv'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

/home/han/anaconda3/envs/ts_cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/ts_cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/ts_cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/ts_cpu/lib/python3.6/site-pack

In [2]:
# Note that x is a 2D numpy array, 
# x[i, :] contains the user-item pair, and y[i] is the corresponding rating.

df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
df_train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name[:-3] + 'pkl'))

num_users = len(df['uid'].unique())
num_tracks = len(df['tid'].unique())
print(num_users, num_tracks)

953 157567


In [3]:
config = Config()
config.num_users = len(df['uid'].unique())
config.num_items = len(df['tid'].unique())
config.min_value = df['rating'].min()
config.max_value = df['rating'].max()

In [4]:
# # Count POP to DF
# tid_list = []
# pop_list = []
# for i in df['tid'].unique():
#     tid_list.append(i)
#     pop_list.append(len(df[df['tid']==i])/config.num_users)

In [5]:
# pre = 1.0
# d = {'tid': tid_list, 'pop': pop_list}
# df_pop = pd.DataFrame(data=d)
# df_pop = df_pop.sort_values(by=['pop'], ascending=False)
# df_pop = df_pop[:int(len(df_pop)*pre)]
# pre_list = df_pop['tid'].tolist()
# len(pre_list)

In [6]:
# # Count POP to DF (according to listen count)
file_name_pop = 'normalized_popularity_filter_track_5_user_100.pkl'
pop_count = pd.read_pickle(os.path.join(dir_, file_name_pop))
pop_count = pop_count.sort_values(by=['count'],  ascending=False)

In [7]:
pre = .6
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized = pop_normalized[:int(len(pop_normalized)*pre)]
pre_list = pop_normalized['tid'].tolist()
len(pre_list)

94540

In [8]:
pre_df_train = df_train[df_train['tid'].isin(pre_list)]
x_train = []
y_train = []
pbar = tqdm(total = len(pre_df_train))
for index, row in pre_df_train.iterrows():
    x_train.append([row['uid'], row['tid']])
    y_train.append(row['rating'])
    pbar.update(1)
pbar.close()

In [9]:
all_tracks = pd.DataFrame()
all_tracks['tid'] = pre_list
all_tracks['count'] = 0

In [ ]:
x_test = []

for i in tqdm(range(num_users)):
    
    user = pre_df_train[pre_df_train['uid']==i]
    top_n = all_tracks.set_index('tid').add(user.set_index('tid'), fill_value=0).reset_index()
    top_n = top_n[top_n['count']==0]
    top_n['uid'] = i
    top_n = top_n[['uid', 'tid']]
    top_n = top_n.values.tolist()
    x_test.extend(top_n)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
print(x_test[0].shape)

In [ ]:
with tf.compat.v1.Session() as sess:
    # For SVD++ algorithm, if `dual` is True, then the dual term of items' 
    # implicit feedback will be added into the original SVD++ algorithm.
    # model = SVDPP(config, sess, dual=False)
    # model = SVDPP(config, sess, dual=True)
    
    model = SVD(config, sess)
    
#     model.train(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=1024)
    model.train(x_train, y_train, epochs=2, batch_size=1024)
    
    y_pred_1 = model.predict(x_test[:int(len(x_test)/2)])
    y_pred_2 = model.predict(x_test[int(len(x_test)/2):])
#     print('rmse: {}, mae: {}'.format(rmse(y_test, y_pred), mae(y_test, y_pred)))

In [ ]:
y_pred = np.append(y_pred_1, y_pred_2)
print(len(y_pred_1), len(y_pred_2), y_pred_1[0], y_pred_2[-1])
print(len(y_pred), y_pred[0], y_pred[-1])

In [ ]:
df = pd.DataFrame(y_pred, columns=['rating'])
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', y_pred, False) 
df2[:10]

In [ ]:
df2.to_pickle(os.path.join(dir_, 'pre_filtering (count)', str(pre) + '_prediction_svd_top_N_' + file_name[:-3] + 'pkl'))